In [7]:
import sys
import os
import pickle as pkl
import pandas as pd
import numpy as np

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

from src.protein_graph import pncaGraph
from src.run_model import pnca_GCN_vary_graph

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%aimport src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
train_seqs = pd.read_csv('../data/real_train_sequences.csv')
test_seqs = pd.read_csv('../data/real_test_sequences.csv')

In [9]:
pnca = pncaGraph(pdb='../pdb/3PL1-PZA.pdb',
# pnca = pncaGraph(pdb='../pdb/test_structures/0_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
                    lig_resname='PZA', 
                    self_loops=False,
                    # lig_pdb='../pdb/3PL1-PZA.pdb',
                    cutoff_distance=8)

In [10]:
# pnca_m = pncaGraph(pdb='../pdb/0_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
#                     lig_resname='PZA', 
#                     self_loops=False,   
#                     cutoff_distance=8)

# pnca_m = pncaGraph(pdb='../dd_pnca/merged.pdb',
#                     lig_resname='PZA', 
#                     self_loops=False,   
#                     cutoff_distance=8)

pnca_m = pncaGraph(pdb='../pdb/test_structures/aligned/pza_attached/99_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
                    lig_resname='PZA', 
                    self_loops=False,   
                    cutoff_distance=8)
pnca_m = pncaGraph(pdb='../dd_pnca/test_structs/99_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
                    lig_resname='PZA', 
                    self_loops=False,   
                    cutoff_distance=8)

FileNotFoundError: [Errno 2] No such file or directory: '../pdb/test_structures/aligned/pza_attached/99_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb'

In [ ]:
pnca_m.gen_dataset(
        wt_seq='',
        sequences= test_seqs.iloc[[0]],
        edge_weights= 'none',
        normalise=True
        )

### Create graphs and corresponding Data objects

In [11]:
def create_graphs(structs_path, seqs):
    
    output_dict = {}
    
    for f in tqdm(os.listdir(structs_path)):
        
        index = f[:f.find('_')]
        name = 'pnca_mut_' + index

        pnca_m = pncaGraph(pdb=f'{structs_path}/{f}',
                        lig_resname='PZA', 
                        self_loops=False,
                        cutoff_distance=8)
        
        metadata = seqs.iloc[[int(index)]]
        # display(metadata)

        output_dict[name] = {
            'graph':pnca_m, 
            'metadata':metadata
            }
        
    return output_dict

In [12]:
test_structs = '../pdb/test_pza'
train_structs = '../pdb/train_pza'
# test_structs = '../pdb/test_structures/aligned/pza_attached'
# train_structs = '../pdb/train_structures/aligned/pza_attached'
train_graph_dict = create_graphs(train_structs, train_seqs)
test_graph_dict = create_graphs(test_structs, test_seqs)

 33%|███▎      | 153/464 [00:19<00:39,  7.95it/s]


KeyboardInterrupt: 

In [ ]:
# structs = '../pdb/test_structures'
# test_graph_dict = {}

# for f in tqdm(os.listdir(structs)):
    
#     index = f[:f.find('_')]
#     name = 'pnca_mut_' + index
#     f_path = f''
#     pnca_m = pncaGraph(pdb=f'../pdb/test_structures/{f}',
#                     lig_resname='PZA', 
#                     self_loops=False,
#                     lig_pdb='../pdb/3PL1-PZA.pdb',
#                     cutoff_distance=8)
    
#     metadata = test_seqs.iloc[[int(index)]]

#     test_graph_dict[name] = {
#         'graph':pnca_m, 
#         'metadata':metadata
#         }

In [ ]:
wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'
# wt_seq = ''
# attach sequence from fast to each key in dictionary
# iterate through dict and creat one sample with gen_dataset per graph
# assign dataset object to dictionary
# will need to fix how to do CLAV dist


for sample in tqdm(test_graph_dict):
    test_graph_dict[sample]['graph'].gen_dataset(
        wt_seq=wt_seq,
        sequences= test_graph_dict[sample]['metadata'],
        edge_weights= 'none',
        normalise=True
        )

for sample in tqdm(train_graph_dict):
    
    train_graph_dict[sample]['graph'].gen_dataset(
        wt_seq=wt_seq,
        sequences= train_graph_dict[sample]['metadata'],
        edge_weights= 'none',
        normalise=True
        )

100%|██████████| 464/464 [01:38<00:00,  4.72it/s]


In [ ]:
for sample in test_graph_dict:
    print(test_graph_dict[sample]['graph'].dataset[0])
    print(test_graph_dict[sample])
    break

Data(x=[185, 16], edge_index=[2, 1840], y=1)
{'graph': <src.protein_graph.pncaGraph object at 0x2f5c878f0>, 'metadata':    phenotype_label  number_resistant_mutations  number_susceptible_mutations  \
99               R                           1                             0   

                                               allele mutation  
99  MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...     I90T  }


In [ ]:
graph_dict = {
    'train': train_graph_dict,
    'test': test_graph_dict
}

In [8]:
# save graph_dict as pickle

# with open('datasets/singletons_af_graph_dict.pkl', 'wb') as f:
#     pkl.dump(graph_dict, f)

with open('datasets/singletons_af_w_pza_graph_dict.pkl', 'wb') as f:
    pkl.dump(graph_dict, f)
    
# with open('datasets/singletons_af_no_mut_feats_graph_dict.pkl', 'wb') as f:
#     pkl.dump(graph_dict, f)

In [20]:
# open graph_dict pickle as dictionary

# with open('datasets/singletons_af_graph_dict.pkl', 'rb') as f:
#     graph_dict = pkl.load(f)

with open('datasets/singletons_af_w_pza_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)

### Inspect and test

In [15]:
from pprint import pprint

pprint(graph_dict)

{'test': {'pnca_mut_0': {'graph': <src.protein_graph.pncaGraph object at 0x2eec18f50>,
                         'metadata':   phenotype_label  number_resistant_mutations  number_susceptible_mutations  \
0               R                           1                             0   

                                              allele mutation  
0  MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...    A143D  },
          'pnca_mut_1': {'graph': <src.protein_graph.pncaGraph object at 0x2e8f34c20>,
                         'metadata':   phenotype_label  number_resistant_mutations  number_susceptible_mutations  \
1               S                           0                             1   

                                              allele mutation  
1  MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...    A143V  },
          'pnca_mut_10': {'graph': <src.protein_graph.pncaGraph object at 0x2fa91d400>,
                          'metadata':    phenotype_label  number_resistant_mutations  

In [21]:
for sample_set in graph_dict:
    print(sample_set)
    for sample in graph_dict[sample_set]:
        print(sample)
        print(graph_dict[sample_set][sample]['graph'])
        display(graph_dict[sample_set][sample]['metadata'])
        break
    break

train
pnca_mut_315


,phenotype_label,number_resistant_mutations,number_susceptible_mutations,allele,mutation
315,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,R148S


In [23]:
a = pnca_GCN_vary_graph(
    self_loops= False,
    cutoff_distance= 5,
    edge_weight_func= '1-(dist/cutoff)',
    batch_size= 1,
    num_node_features = 1,
    hidden_channels = 1,
    learning_rate = 1,
    wd = 1,
    epochs= 1,
    graph_dict= graph_dict,
    output_channels= 2,
    normalise_ews = True,
    dropout = 0.5,
    lr_scheduling = False)

Adjusting edge index and attaching edge weights for cutoff distance 5


In [24]:
a

[Data(x=[185, 16], edge_index=[2, 306], y=0, edge_attr=[306, 1]),
 Data(x=[185, 16], edge_index=[2, 312], y=1, edge_attr=[312, 1]),
 Data(x=[185, 16], edge_index=[2, 318], y=1, edge_attr=[318, 1]),
 Data(x=[185, 16], edge_index=[2, 314], y=0, edge_attr=[314, 1]),
 Data(x=[185, 16], edge_index=[2, 316], y=0, edge_attr=[316, 1]),
 Data(x=[185, 16], edge_index=[2, 318], y=0, edge_attr=[318, 1]),
 Data(x=[185, 16], edge_index=[2, 316], y=0, edge_attr=[316, 1]),
 Data(x=[185, 16], edge_index=[2, 330], y=0, edge_attr=[330, 1]),
 Data(x=[185, 16], edge_index=[2, 322], y=0, edge_attr=[322, 1]),
 Data(x=[185, 16], edge_index=[2, 314], y=1, edge_attr=[314, 1]),
 Data(x=[185, 16], edge_index=[2, 312], y=1, edge_attr=[312, 1]),
 Data(x=[185, 16], edge_index=[2, 316], y=1, edge_attr=[316, 1]),
 Data(x=[185, 16], edge_index=[2, 324], y=1, edge_attr=[324, 1]),
 Data(x=[185, 16], edge_index=[2, 322], y=0, edge_attr=[322, 1]),
 Data(x=[185, 16], edge_index=[2, 314], y=1, edge_attr=[314, 1]),
 Data(x=[1

In [95]:
# import torch
# from sklearn.preprocessing import MinMaxScaler
# # normalise, column wise for entire dataset
# all_features = torch.cat([data.x for data in a], dim=0)
# scaler = MinMaxScaler()
# scaler.fit(all_features.numpy())

# # Apply normalization to each graph
# for data in a:
#     data.x = torch.tensor(scaler.transform(data.x.numpy()), dtype=torch.float)

In [9]:
all_features.shape

torch.Size([122840, 16])

In [25]:
df = pd.DataFrame(a[20].x)
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.612999,0.114907,0.573974,0.371715,0.711111,0.0,0.0,0.0,0.944901,0.498856,0.829897,0.294140,0.0,0.735926,0.132463,0.0
1,0.675611,0.562112,0.767622,1.000000,0.000000,0.0,1.0,0.0,0.812253,0.195479,0.876056,0.204318,0.0,0.735926,0.132463,0.0
2,0.169946,0.354037,0.108443,0.404255,0.700000,0.0,0.0,0.0,0.676916,0.163494,0.910516,0.000000,0.0,0.735926,0.132463,0.0
3,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.557315,0.225677,0.858576,0.000000,0.0,0.735926,0.132463,0.0
4,0.635659,0.000000,0.434547,0.406758,1.000000,0.0,0.0,0.0,0.405090,0.197237,0.815362,0.000000,0.0,0.735926,0.132463,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.466905,0.751553,0.557707,0.056320,0.111111,1.0,0.0,0.0,0.688903,0.277505,0.835621,0.436392,0.0,0.735926,0.132463,0.0
181,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.632050,0.233687,0.828841,0.097918,0.0,0.735926,0.132463,0.0
182,0.476446,0.086957,0.325329,0.399249,0.966667,0.0,0.0,0.0,0.705192,0.115015,0.936444,0.119507,0.0,0.735926,0.132463,0.0
183,0.288611,0.319876,0.357088,0.287860,0.777778,0.0,0.0,0.0,0.719099,0.247083,0.911766,0.269468,0.0,0.735926,0.132463,0.0


In [99]:
df = pd.DataFrame(a[20].x)
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.612999,0.114907,0.573974,0.371715,0.711111,0.0,0.0,0.0,0.966502,0.498758,0.829785,0.292131,0.0,0.735926,0.132436,0.0
1,0.675611,0.562112,0.767622,1.000000,0.000000,0.0,1.0,0.0,0.837465,0.195462,0.876000,0.196545,0.0,0.735926,0.132436,0.0
2,0.169946,0.354037,0.108443,0.404255,0.700000,0.0,0.0,0.0,0.697541,0.163481,0.910571,0.000000,0.0,0.735926,0.132436,0.0
3,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.574762,0.225630,0.858604,0.001536,0.0,0.735926,0.132436,0.0
4,0.635659,0.000000,0.434547,0.406758,1.000000,0.0,0.0,0.0,0.414666,0.197165,0.815390,0.000000,0.0,0.735926,0.132436,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.466905,0.751553,0.557707,0.056320,0.111111,1.0,0.0,0.0,0.710247,0.277510,0.835677,0.432246,0.0,0.735926,0.132436,0.0
181,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.652160,0.233723,0.828813,0.092131,0.0,0.735926,0.132436,0.0
182,0.476446,0.086957,0.325329,0.399249,0.966667,0.0,0.0,0.0,0.737158,0.114810,0.936388,0.116315,0.0,0.735926,0.132436,0.0
183,0.288611,0.319876,0.357088,0.287860,0.777778,0.0,0.0,0.0,0.743776,0.247035,0.911850,0.274856,0.0,0.735926,0.132436,0.0


In [81]:
import torch
torch.isnan(a[0].x).any()

tensor(False)

In [96]:
df = pd.DataFrame(a[20].x)
display(df)
df[df.isna().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.612999,0.114907,0.573974,0.371715,0.711111,0.0,0.0,0.0,0.966502,0.498758,0.829785,0.292131,0.0,0.735926,0.132436,0.0
1,0.675611,0.562112,0.767622,1.000000,0.000000,0.0,1.0,0.0,0.837465,0.195462,0.876000,0.196545,0.0,0.735926,0.132436,0.0
2,0.169946,0.354037,0.108443,0.404255,0.700000,0.0,0.0,0.0,0.697541,0.163481,0.910571,0.000000,0.0,0.735926,0.132436,0.0
3,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.574762,0.225630,0.858604,0.001536,0.0,0.735926,0.132436,0.0
4,0.635659,0.000000,0.434547,0.406758,1.000000,0.0,0.0,0.0,0.414665,0.197165,0.815390,0.000000,0.0,0.735926,0.132436,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.466905,0.751553,0.557707,0.056320,0.111111,1.0,0.0,0.0,0.710247,0.277510,0.835677,0.432246,0.0,0.735926,0.132436,0.0
181,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.652159,0.233723,0.828813,0.092131,0.0,0.735926,0.132436,0.0
182,0.476446,0.086957,0.325329,0.399249,0.966667,0.0,0.0,0.0,0.737158,0.114810,0.936388,0.116315,0.0,0.735926,0.132436,0.0
183,0.288611,0.319876,0.357087,0.287860,0.777778,0.0,0.0,0.0,0.743776,0.247035,0.911850,0.274856,0.0,0.735926,0.132436,0.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15


In [33]:
graph_dict['train']['pnca_mut_99']

{'graph': <src.protein_graph.pncaGraph at 0x2e5d46360>,
 'metadata':    phenotype_label  number_resistant_mutations  number_susceptible_mutations  \
 99               R                           1                             0   
 
                                                allele mutation  
 99  MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...     D63E  }

In [26]:
df.iloc[[147]]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
147,0.172332,0.354037,0.232378,0.364205,0.411111,0.5,0.2,0.0,0.642102,NaN,NaN,NaN,1.0,0.0,1.0,1.0


In [39]:
import sbmlcore
sbmlcore.Stride('../pdb/train_structures/99_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb', dataset_type='amino_acid').results.iloc[[62]]

,resid,amino_acid,segid,secondary_structure,secondary_structure_long,phi,psi,residue_sasa,n_hbond_acceptors,n_hbond_donors,B,C,E,G,H,T
62,63,E,A,T,Turn,-86.16,-20.01,130.5,0.0,0.0,False,False,False,False,False,True


In [72]:
graph_dict['train']['pnca_mut_315']['metadata'].allele.values[0]

'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVSNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCSS!'

In [69]:
sd = sbmlcore.StructuralDistances('../pdb/3PL1-PZA.pdb', 'resname PZA', 'PZA_dist',dataset_type='amino_acid', infer_masses=False).results

In [74]:
sd['amino_acid'] = list(graph_dict['train']['pnca_mut_315']['metadata'].allele.values[0][:185])

### Try using PDB of AlphaFold structure with PZA inserted by structural alignment

In [50]:
pnca_m = pncaGraph(pdb='../pdb/train_structures/0_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
                    lig_resname='PZA', 
                    self_loops=False,   
                    cutoff_distance=8)

In [51]:
pnca_m_pza = pncaGraph(pdb='../pdb/af_pnca_align_test.pdb',
                    lig_resname='PZA', 
                    self_loops=False,   
                    cutoff_distance=8)

In [10]:
pnca_m.d_array

array([[ 0.        ,  8.20326466,  7.03619389, ..., 18.70025333,
        23.08515348, 19.66126558],
       [ 8.20326466,  0.        ,  6.5498121 , ..., 10.537969  ,
        15.28131748, 12.89503302],
       [ 7.03619389,  6.5498121 ,  0.        , ..., 14.8002032 ,
        18.67954176, 15.25503806],
       ...,
       [18.70025333, 10.537969  , 14.8002032 , ...,  0.        ,
         5.83841433,  7.74490483],
       [23.08515348, 15.28131748, 18.67954176, ...,  5.83841433,
         0.        ,  5.76031273],
       [19.66126558, 12.89503302, 15.25503806, ...,  7.74490483,
         5.76031273,  0.        ]])

In [20]:
pnca_m_pza.d_array

array([[ 0.        ,  8.2030901 ,  7.03615553, ..., 18.7001058 ,
        23.08542588, 19.66143269],
       [ 8.2030901 ,  0.        ,  6.54968463, ..., 10.53799167,
        15.28174892, 12.8953889 ],
       [ 7.03615553,  6.54968463,  0.        , ..., 14.80016558,
        18.67984247, 15.2551938 ],
       ...,
       [18.7001058 , 10.53799167, 14.80016558, ...,  0.        ,
         5.83861822,  7.74501105],
       [23.08542588, 15.28174892, 18.67984247, ...,  5.83861822,
         0.        ,  5.76061418],
       [19.66143269, 12.8953889 , 15.2551938 , ...,  7.74501105,
         5.76061418,  0.        ]])

In [22]:
pd.DataFrame(pnca_m.d_array - pnca_m_pza.d_array).describe()

,0,1,2,3,4,5,6,7,8,9,...,175,176,177,178,179,180,181,182,183,184
count,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,...,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000
mean,0.000020,0.000051,-0.000021,0.000095,0.000108,-0.000008,-0.000022,-0.000019,-0.000109,0.000109,...,0.000067,-0.000114,-0.000122,0.000027,0.000004,0.000018,0.000006,-0.000053,-0.000145,-0.000083
std,0.000204,0.000221,0.000170,0.000267,0.000266,0.000257,0.000172,0.000195,0.000219,0.000237,...,0.000173,0.000211,0.000193,0.000198,0.000206,0.000190,0.000197,0.000187,0.000185,0.000186
min,-0.000534,-0.000490,-0.000481,-0.000638,-0.000523,-0.000639,-0.000507,-0.000533,-0.000677,-0.000816,...,-0.000443,-0.000679,-0.000673,-0.000614,-0.000537,-0.000509,-0.000465,-0.000538,-0.000594,-0.000525
25%,-0.000111,-0.000092,-0.000123,-0.000046,-0.000085,-0.000174,-0.000124,-0.000137,-0.000249,-0.000041,...,-0.000035,-0.000252,-0.000233,-0.000092,-0.000144,-0.000100,-0.000125,-0.000181,-0.000266,-0.000210
50%,0.000033,0.000068,-0.000022,0.000133,0.000125,-0.000006,-0.000018,-0.000024,-0.000113,0.000127,...,0.000081,-0.000100,-0.000106,0.000014,-0.000013,0.000023,0.000012,-0.000061,-0.000143,-0.000087
75%,0.000157,0.000216,0.000099,0.000259,0.000297,0.000158,0.000091,0.000105,0.000029,0.000277,...,0.000178,0.000050,-0.000002,0.000157,0.000159,0.000133,0.000136,0.000087,-0.000033,0.000029
max,0.000576,0.000640,0.000488,0.000757,0.000651,0.000669,0.000463,0.000464,0.000444,0.000761,...,0.000560,0.000329,0.000280,0.000512,0.000492,0.000434,0.000588,0.000386,0.000426,0.000336


In [27]:
pnca_m_pza.nodes.residues.resnames

array(['MET', 'ARG', 'ALA', 'LEU', 'ILE', 'ILE', 'VAL', 'ASP', 'VAL',
       'GLN', 'ASN', 'ASP', 'PHE', 'CYS', 'GLU', 'GLY', 'GLY', 'SER',
       'LEU', 'ALA', 'VAL', 'THR', 'GLY', 'GLY', 'ALA', 'ALA', 'LEU',
       'ALA', 'ARG', 'ALA', 'ILE', 'SER', 'ASP', 'TYR', 'LEU', 'ALA',
       'GLU', 'ALA', 'ALA', 'ASP', 'TYR', 'HIS', 'HIS', 'VAL', 'VAL',
       'ALA', 'THR', 'LYS', 'ASP', 'PHE', 'HIS', 'ILE', 'ASP', 'PRO',
       'GLY', 'ASP', 'HIS', 'PHE', 'SER', 'GLY', 'THR', 'PRO', 'ASP',
       'TYR', 'SER', 'SER', 'SER', 'TRP', 'PRO', 'PRO', 'HIS', 'CYS',
       'VAL', 'SER', 'GLY', 'THR', 'PRO', 'GLY', 'ALA', 'ASP', 'PHE',
       'HIS', 'PRO', 'SER', 'LEU', 'ASP', 'THR', 'SER', 'ALA', 'ILE',
       'GLU', 'ALA', 'VAL', 'PHE', 'TYR', 'LYS', 'GLY', 'ALA', 'TYR',
       'THR', 'GLY', 'VAL', 'TYR', 'SER', 'GLY', 'PHE', 'GLU', 'GLY',
       'VAL', 'ASP', 'GLU', 'ASN', 'GLY', 'THR', 'PRO', 'LEU', 'LEU',
       'ASN', 'TRP', 'LEU', 'ARG', 'GLN', 'ARG', 'GLY', 'VAL', 'ASP',
       'GLU', 'VAL',

In [29]:
aa_dict = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
    'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V',
    'TRP': 'W', 'TYR': 'Y'
}

sequence = ''.join([aa_dict[res] for res in pnca_m_pza.nodes.residues.resnames])

In [30]:
sequence

'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGVYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [35]:
train_seqs.iloc[0].allele[:185] == sequence

True

In [53]:
pnca_m_pza.gen_dataset(
        wt_seq='XXXXXX',
        sequences= train_seqs.iloc[[0]],
        edge_weights= 'none',
        normalise=True
        )
wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'
# pnca_m.gen_dataset(
#         wt_seq= wt_seq,
#         sequences= train_seqs.iloc[[0]],
#         edge_weights= 'none',
#         normalise=True
#         )

[autoreload of src.protein_graph failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 860, in _exec
  File "<frozen importlib._bootstrap_external>", line 994, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN/src/protein_graph.py", line 10, in <module>
    from model_helpers import convert_3lett

In [42]:
pd.DataFrame(pnca_m_pza.dataset[0].x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,162.899994,-0.44,149.199997,5.74,1.9,0.0,0.0,0.0,27.694527,0.000000,118.589996,70.099998,-100.0,0.0,0.0,0.0
1,173.399994,1.00,174.199997,10.76,-4.5,0.0,5.0,0.0,24.580061,-107.570000,135.960007,53.799999,-100.0,0.0,0.0,0.0
2,88.599998,0.33,89.099998,6.00,1.8,0.0,0.0,0.0,21.396559,-120.139999,147.009995,0.000000,-100.0,0.0,0.0,0.0
3,166.699997,-0.69,131.199997,5.98,3.8,0.0,0.0,0.0,18.572495,-97.379997,129.289993,0.200000,-100.0,0.0,0.0,0.0
4,166.699997,-0.81,131.199997,6.02,4.5,0.0,0.0,0.0,15.006265,-108.580002,113.660004,0.000000,-100.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,138.399994,1.61,147.100006,3.22,-3.5,4.0,0.0,0.0,21.576748,-79.180000,120.540001,113.000000,-100.0,0.0,0.0,0.0
181,166.699997,-0.69,131.199997,5.98,3.8,0.0,0.0,0.0,20.233185,-94.540001,118.000000,30.100000,-100.0,0.0,0.0,0.0
182,140.000000,-0.53,117.099998,5.96,4.2,0.0,0.0,0.0,21.967924,-136.639999,155.809998,29.900000,-100.0,0.0,0.0,0.0
183,108.500000,0.22,121.199997,5.07,2.5,0.0,0.0,0.0,22.289364,-90.870003,144.470001,89.900002,-100.0,0.0,0.0,0.0


In [46]:
pd.DataFrame(graph_dict['train']['pnca_mut_0']['graph'].dataset[0].x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.612999,0.114907,0.573974,0.371715,0.711111,0.0,0.0,0.0,0.966502,0.498758,0.829369,0.274856,0.0,0.735926,0.132436,0.0
1,0.675611,0.562112,0.767622,1.000000,0.000000,0.0,1.0,0.0,0.837465,0.198755,0.877807,0.202687,0.0,0.735926,0.132436,0.0
2,0.169946,0.354037,0.108443,0.404255,0.700000,0.0,0.0,0.0,0.697541,0.163453,0.908515,0.000000,0.0,0.735926,0.132436,0.0
3,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.574762,0.227053,0.859215,0.001536,0.0,0.735926,0.132436,0.0
4,0.635659,0.000000,0.434547,0.406758,1.000000,0.0,0.0,0.0,0.414666,0.195797,0.815863,0.000000,0.0,0.735926,0.132436,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.466905,0.751553,0.557707,0.056320,0.111111,1.0,0.0,0.0,0.710247,0.277705,0.835010,0.433397,0.0,0.735926,0.132436,0.0
181,0.635659,0.037267,0.434547,0.401752,0.922222,0.0,0.0,0.0,0.652160,0.234839,0.828007,0.115931,0.0,0.735926,0.132436,0.0
182,0.476446,0.086957,0.325329,0.399249,0.966667,0.0,0.0,0.0,0.737158,0.117378,0.933054,0.113244,0.0,0.735926,0.132436,0.0
183,0.288611,0.319876,0.357088,0.287860,0.777778,0.0,0.0,0.0,0.743776,0.245053,0.901651,0.350096,0.0,0.735926,0.132436,0.0
